# KPI Pipeline
This notebook ingests `data/sample/sample_ops_tickets.csv`, computes weekly KPIs, and writes `data/processed/weekly_kpis.csv`.
Use this as a starting point; adapt metric definitions as needed.

In [ ]:
import pandas as pd
from pathlib import Path
src = Path('../data/sample/sample_ops_tickets.csv')
df = pd.read_csv(src, parse_dates=['created_at','closed_at'])

# Derived fields
df['resolution_hrs'] = (df['closed_at'] - df['created_at']).dt.total_seconds() / 3600
df['breach'] = (df['resolution_hrs'] > df['sla_target_hrs']).astype(int)
df['week'] = df['created_at'].dt.to_period('W').dt.start_time

weekly = df.groupby('week').agg(
    tickets=('tid','count'),
    avg_resolution_hrs=('resolution_hrs','mean'),
    breach_rate=('breach','mean')
).reset_index()

out = Path('../data/processed/weekly_kpis.csv')
out.parent.mkdir(parents=True, exist_ok=True)
weekly.to_csv(out, index=False)
weekly.head()

### Next steps
- Connect `data/processed/weekly_kpis.csv` in Power BI; create measures for On-Time %, Avg Handle Time, Backlog EoD.
- Add DAX measures and visuals; export screenshots to `/images/` and the `.pbix` to `/powerbi/`. 